# Лабораторная 6: Retrieval-Augmented Generation (RAG)

Минимальный пример RAG для корпоративной базы знаний на `wiki.txt`. Кандидатогенерация — локально (RuBERT-tiny), генерация — через бесплатные модели OpenRouter.

**Перед запуском**:
1. Установите зависимости (`pip install -q sentence-transformers faiss-cpu openai tiktoken`).
2. Определите переменную окружения `OPENROUTER_API_KEY` (получить ключ в профиле OpenRouter).
3. Проверьте, что файл `wiki.txt` лежит рядом с ноутбуком.

Изменения для более релевантных ответов:
- нормализация эмбеддингов + cosine similarity (FAISS IndexFlatIP);
- лёгкий лексический буст при ранжировании (совпадение токенов запроса);
- более строгий промпт и форматированный вывод кандидатов/ответа.



In [1]:
# Установка библиотек (один раз)
!pip install -q sentence-transformers faiss-cpu openai tiktoken rich

In [ ]:
import os
import re
from typing import List, Tuple

import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from openai import OpenAI
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich import box
from rich.markdown import Markdown

console = Console()

/home/ameno/Work/University/Методы и задачи обработки естественных языков/LAB6/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Путь к базе знаний
WIKI_PATH = "wiki.txt"
assert os.path.exists(WIKI_PATH), "Файл wiki.txt не найден"

with open(WIKI_PATH, "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"Длина базы знаний (символов): {len(raw_text):,}")

Длина базы знаний (символов): 3,809


In [ ]:
# Чанкер по разделам регламента (1., 2., 3., ...), чтобы не ломать смысл
import re

def chunk_text(text: str) -> List[str]:
    lines = [ln.rstrip() for ln in text.split('\n')]
    chunks: List[str] = []
    buf: List[str] = []
    for ln in lines:
        if re.match(r'^\d+\.\s', ln) and buf:
            chunks.append('\n'.join([x for x in buf if x]))
            buf = [ln]
        else:
            buf.append(ln)
    if buf:
        chunks.append('\n'.join([x for x in buf if x]))
    # чистим пустые
    return [c for c in chunks if c.strip()]


chunks = chunk_text(raw_text)
print(f"Число чанков: {len(chunks)}")
for i, c in enumerate(chunks, 1):
    print(f"\n--- Чанк {i} ---\n{c}")

Число чанков: 8

--- Чанк 1 ---
УТВЕРЖДЕНО
Верховным Алгоритмом
от 32 мартобря 2077 года
РЕГЛАМЕНТ ВНУТРЕННЕГО ТРУДОВОГО РАСПОРЯДКА
ООО «КИБЕР-ЧЕБУРЕКИ» (Версия 42.0-beta)

--- Чанк 2 ---
1. ОБЩИЕ ПОЛОЖЕНИЯ
1.1. Настоящий регламент регулирует поведение биологических и полубиологических единиц (далее — Сотрудники) в офисах и на орбитальных станциях Компании.
1.2. Каждый сотрудник обязан носить идентификационный чип в левом ухе. В случае потери уха в производственной дуэли, чип переносится в нос или иную выступающую часть тела.
1.3. Рабочий день начинается, когда индикатор биоритмов Директора загорается зеленым, и заканчивается, когда уровень кислорода в офисе падает до 15%.
1.4. Незнание законов робототехники не освобождает от ответственности перед лазерным шредером.

--- Чанк 3 ---
2. БЕЗОПАСНОСТЬ И ДОСТУП
2.1. При обнаружении в столовой робота-шпиона модели Т-800, запрещается вступать с ним в философские споры о смысле жизни. Следует немедленно предложить ему смазку WD-40 и вызвать IT

In [ ]:
# Инициализация эмбеддера (RuBERT-tiny) и построение FAISS-индекса (cosine similarity)
model_name = "cointegrated/rubert-tiny2"
embedder = SentenceTransformer(model_name)

# Нормализуем эмбеддинги и используем IndexFlatIP для косинусной близости
embeddings = embedder.encode(
    chunks,
    convert_to_numpy=True,
    show_progress_bar=True,
    batch_size=64,
    normalize_embeddings=True,
)
embeddings = embeddings.astype("float32")

index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

print("FAISS индекс готов", index.ntotal)

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.54it/s]

FAISS индекс готов 8


In [ ]:
# Поиск кандидатов с лёгким лексическим бустом

def lexical_overlap_score(text: str, query_tokens: set) -> float:
    tokens = set(re.findall(r"\w+", text.lower()))
    if not query_tokens:
        return 0.0
    return len(tokens & query_tokens) / (len(query_tokens) + 1e-6)


def retrieve_candidates(query: str, top_k: int = 5, search_k: int = 12) -> List[Tuple[str, float, float]]:
    # search_k — сколько кандидатов вытаскиваем из вектора, затем переранжируем
    q_vec = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    q_vec = q_vec.astype("float32")
    sims, idxs = index.search(q_vec, search_k)
    q_tokens = set(re.findall(r"\w+", query.lower()))

    scored = []
    for sim, idx in zip(sims[0], idxs[0]):
        ctx = chunks[idx]
        overlap = lexical_overlap_score(ctx, q_tokens)
        combined = 0.7 * float(sim) + 0.3 * overlap
        scored.append((combined, float(sim), float(overlap), ctx))

    scored.sort(key=lambda x: x[0], reverse=True)
    return [(ctx, sim, overlap) for combined, sim, overlap, ctx in scored[:top_k]]


# Быстрая проверка поиска
for c, sim, ov in retrieve_candidates("Когда разрешена работа из дома?", top_k=2):
    print(f"sim={sim:.4f}, overlap={ov:.3f}\n{c[:300]}\n---")

sim=0.5118, overlap=0.800
5. УДАЛЕННАЯ РАБОТА (НЕЙРО-СВЯЗЬ)
5.1. Работа из дома разрешена только при наличии стабильного нейро-канала скоростью не менее 10 Терабит/с.
5.2. Если во время зум-колла у сотрудника на фоне появляется кот, кот обязан быть одет в корпоративный галстук. Коты без галстуков считаются посторонними агент
---
sim=0.4507, overlap=0.400
2. БЕЗОПАСНОСТЬ И ДОСТУП
2.1. При обнаружении в столовой робота-шпиона модели Т-800, запрещается вступать с ним в философские споры о смысле жизни. Следует немедленно предложить ему смазку WD-40 и вызвать IT-шамана.
2.2. Пароль от корпоративного Wi-Fi "Skynet_Guest" меняется каждый раз, когда курс б
---


In [7]:
# Клиент OpenRouter (нужен OPENROUTER_API_KEY)
api_key = os.getenv("OPENROUTER_API_KEY")
if not api_key:
    raise RuntimeError("Установите переменную окружения OPENROUTER_API_KEY")

client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)
DEFAULT_MODEL = "amazon/nova-2-lite-v1:free"  # бесплатная модель


def generate_answer(question: str, context_docs: List[str], model: str = DEFAULT_MODEL) -> str:
    context_text = "\n\n".join(context_docs)
    system_prompt = (
        "Ты — умный ассистент корпоративной базы знаний. "
        "Отвечай ТОЛЬКО на основе предоставленного контекста. "
        "Если информации нет, отвечай: 'Я не знаю, в базе данных этого нет'. "
        "Формат ответа: кратко по делу, 1-2 абзаца или маркированный список."
    )
    user_prompt = f"Контекст:\n{context_text}\nВопрос:\n{question}"

    response = client.chat.completions.create(
        model=model,
        max_tokens=400,
        temperature=0.1,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
    )
    return response.choices[0].message.content

In [8]:
# Основная функция: поиск кандидатов + генерация ответа (rich-вывод)

def answer_question(question: str, top_k: int = 4, model: str = DEFAULT_MODEL) -> str:
    candidates = retrieve_candidates(question, top_k=top_k)
    selected_contexts = [c for c, _, _ in candidates]

    table = Table(title="Топ кандидатов", box=box.SIMPLE_HEAVY, show_lines=False)
    table.add_column("#", style="cyan", justify="right", width=3)
    table.add_column("sim", style="green")
    table.add_column("overlap", style="magenta")
    table.add_column("фрагмент", style="white", overflow="fold")

    for i, (ctx, sim, ov) in enumerate(candidates, 1):
        snippet = ctx.replace("\n", " ")
        if len(snippet) > 400:
            snippet = snippet[:397] + "..."
        table.add_row(str(i), f"{sim:.4f}", f"{ov:.3f}", snippet)

    console.rule("Поиск кандидатов", style="cyan")
    console.print(table)

    answer = generate_answer(question, selected_contexts, model=model)
    console.rule("Ответ модели", style="green")
    console.print(Panel(Markdown(answer), border_style="green"))
    return answer


In [9]:
# Пример вопросов. Запускайте ячейку для проверки, не злоупотребляйте квотой OpenRouter.
questions = [
    "Когда выдают бесплатные чебуреки?",
    'К кому нужно обращаться "О, Величайший Калькулятор Судеб"?',
    "Что происходит с сотрудниками спорившими с роботами?",
    "Что делать если найден робот-шпион Т-1000?",
    "Что делать если найден робот-шпион Т-800?"
]

# Пример вызова (раскомментируйте строчки ниже)
for q in questions:
    print(f"\n=== {q} ===")
    answer_question(q, top_k=1)


=== Когда выдают бесплатные чебуреки? ===


──────────────────────────────────────────────── Поиск кандидатов ─────────────────────────────────────────────────

                                                  Топ кандидатов                                                   
                                                                                                                   
    #   sim      overlap   фрагмент                                                                                
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
    1   0.4744   0.500     3. ПИТАНИЕ И СОЦИАЛЬНЫЙ ПАКЕТ 3.1. Бесплатные чебуреки выдаются по пятницам строго с    
                           12:00 до 12:05. Лимит: 3 штуки в одни руки (или манипуляторы). Попытка взять четвертый  
                           чебурек активирует систему тушения пожара. 3.2. Разогрев рыбы в микроволновке в офисе   
                           приравнивается к акту биологического терроризма. Виновный ссылается в Отдел Маркетинга  
                           на рудники Марса сроком на 5 лет без права переписки...                                

────────────────────────────────────────────────── Ответ модели ───────────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Бесплатные чебуреки выдаются по пятницам строго с 12:00 до 12:05. Лимит — 3 штуки на одного человека (или       │
│ манипуляторы) за раз. Попытка взять четвертый чебурек активирует систему тушения пожара.                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...


=== Что делать если найден робот-шпион Т-800? ===


──────────────────────────────────────────────── Поиск кандидатов ─────────────────────────────────────────────────

                                                  Топ кандидатов                                                   
                                                                                                                   
    #   sim      overlap   фрагмент                                                                                
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
    1   0.5725   0.250     2. БЕЗОПАСНОСТЬ И ДОСТУП 2.1. При обнаружении в столовой робота-шпиона модели Т-800,    
                           запрещается вступать с ним в философские споры о смысле жизни. Следует немедленно       
                           предложить ему смазку WD-40 и вызвать IT-шамана. 2.2. Пароль от корпоративного Wi-Fi    
                           "Skynet_Guest" меняется каждый раз, когда курс биткоина падает более чем на 10%.        
                           Текущий пароль можно узнать, принеся жертву (пончик) сисадмину. ...                    

────────────────────────────────────────────────── Ответ модели ───────────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                              Что делать, если найден робот-шпион Т-800 в столовой:                              │
│                                                                                                                 │
│ Согласно пункту 2.1 раздела «БЕЗОПАСНОСТЬ И ДОСТУП**, при обнаружении робота-шпиона модели Т-800 в столовой     │
│ запрещено вступать с ним в философские споры о смысле жизни. Нужно немедленно:                                  │
│                                                                                                                 │
│  1 Предложить ему смазку WD-40.                                                                                 │
│  2 Вызвать IT-шамана.                                                                                           │
│                                                                                                                 │
│ Эти действия являются обязательными мерами безопасности.                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯